## Data Engineering Capstone Project
### Project Summary

In this project, the key objective of this project is create an ETL pipline from  I94 Immigration
is figre out how factors affect the number of tourists,some factors is :
- the temperature
- the reasonality of travel
- the number of entry ports
- the demographics of various citis.

Data can be used to analyse immigration flow to and from US through different airports. It's used a star schema with a facts table an dimensional tables.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
# Do all imports and installs here
import os
import pandas as pd
import configparser
import csv
import geopandas
import matplotlib.pyplot as plt
%matplotlib inline
# For Spark lib
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType as Double, StringType as Str, IntegerType as Int,\
    TimestampType as Timestamp, DateType as Date, LongType as Long
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear
from pyspark.sql.types import *

## Step 1: Scope the Project and Gather Data
### Project Scope 
In this project, I will utilize the PySpark to perform an extract data , transform data and load ETL pipline.
The ETL pipeline is base on the following steps:
    Collect data from Database --> Processing data --> Cleaning data --> Storing data to the Data warehourse.
The output of ETL is Star Schema model to parquet files and it can be store in local system,AWS Redshift,Cloud Database or ..etc..
The main tools is pandas,pyspark 

### Describe and Gather Data 
The project is using the datasets:
 - Immigration Data: A data dictionary is included in the workspace.This data comes from the U.S. National Tourism and Trade Office. More information on the immigration data [here](https://travel.trade.gov/research/reports/i94/historical/2016.html).
 - US Cities Demographic: the data of the demographic of all US cities and census-designated places with a population greater or equal to 65,000. Dataset comes from OpenSoft found [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
 - Global Land Temperatures By City: the data of the temperatures of various cities in the world from  1743 to 2013. This dataset came from Kaggle found [here](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data).
 - Airport Code Table: Airpot codes data contains information about different airports around the world. The data come from [here](https://datahub.io/core/airport-codes#data)

### Load Configuration Data

In [ ]:
config = configparser.ConfigParser()
config.read('AWS.cfg')
DATA_FOLDER = "./data/"
RESULT_FOLDER = "./data/result/"
SAS_LableFile_path = DATA_FOLDER + 'I94_SAS_Labels_Descriptions.SAS'
KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

In [ ]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Immigration Dataset
The immigration dataset is large. Because the project was developed on my PC local so that I already download all data files to store inside the data folder

By using the pyspark python lib, I import the I94 immigration dataset to Spark

In [ ]:
df_immigration_sp=spark.read.parquet("./data/sas_data")
df_immigration_sp.printSchema()
df_immigration_sp.show(5)

### Adding data dictinaries
In this step, I add more datasets from the data dictionary. First, I read the I94_SAS_Labels_Descriptions.SAS file includes complex data such as country name, VISA type, airport name, airport mode,..etc.Therefore, I will run some processes to extract CSV files.

In [ ]:

def extract_data_from_SAS_labels(input_label):
    '''
    A procedure that returns a cleaned list of code data pairs for the provided input label
    Parameters:
        Input:
        input_label : str
            name of the label in the SAS labels descriptions file
        
        Returns:
        code_data_list : list(tuple(str, str))
            a list of code data pairs extracted from the SAS labels descriptions file and cleaned
    '''
    with open(SAS_LableFile_path) as labels_descriptions:
            raw_labels = labels_descriptions.read()
    # extract only label data
    labels = raw_labels[raw_labels.index(input_label):]
    labels = labels[:labels.index(';')]
    # in each line remove unnecessary spaces and extract the code and its corresponding value 
    lines = labels.splitlines()
    code_data_list = []
    # In case the input_label is I94PORT. This is speical dataset need 3 columm.
    if input_label == "I94PORT": 
        for line in lines:
            try:
                code, data = line.split('=')
                code = code.strip().strip("'").strip('"')
                data = data.strip().strip("'").strip('"').strip()
                _value1 , _value2 = data.split(',')
                _value1 = _value1.strip()
                _value2 = _value2.strip()
                code_data_list.append((code,_value1,_value2))
            except:
                pass
    else:
        for line in lines:
            try:
                code, data = line.split('=')
                code = code.strip().strip("'").strip('"')
                data = data.strip().strip("'").strip('"').strip()
                code_data_list.append((code, data))
            except:
                pass
        
    return code_data_list

The list array the input code,name of output CSV files and definition of header for each CSV files.

In [ ]:
_List_I94_Factor = ["I94CIT & I94RES","I94PORT","I94MODE","I94ADDR","I94BIR"]
_List_CSV_Extract_FileName = ['i94_country.csv','i94_port.csv','i94_model.csv','i94_state_addrl.csv','i94_visa.csv']
_List_CSV_Header = [['code','country_name'],['code','port','state_code'],['code','model'],['code','state'],['code','VISA_Type']]


Read the I94_SAS_Labels_Descriptions.SAS file to extract to CSV files data.

In [ ]:
for index in range(len(_List_I94_Factor)):
    _name_file = DATA_FOLDER + _List_CSV_Extract_FileName[index]
    csvfile = open(_name_file,'w',encoding='UTF8',newline = '')
    writer = csv.writer(csvfile)
    writer.writerow(_List_CSV_Header[index])
    writer.writerows(extract_data_from_SAS_labels(_List_I94_Factor[index]))
    csvfile.close()

By using pandas, I import CSV  files from the database to enrich the data

In [ ]:
df_immig_sample = pd.read_csv(DATA_FOLDER + 'immigration_data_sample.csv')
i94_Country = pd.read_csv (DATA_FOLDER + 'i94_country.csv')
i94_Model= pd.read_csv(DATA_FOLDER + 'i94_model.csv')
i94_State = pd.read_csv(DATA_FOLDER + 'i94_state_addrl.csv')
i94_VISA = pd.read_csv(DATA_FOLDER + 'i94_visa.csv')
df_demographics = pd.read_csv(DATA_FOLDER +'us-cities-demographics.csv', sep=';')
df_ariport_Code = pd.read_csv(DATA_FOLDER +'airport-codes_csv.csv')
df_temperature = pd.read_csv(DATA_FOLDER +'GlobalLandTemperaturesByCity.csv')
i94_Port = pd.read_csv(DATA_FOLDER + 'i94_port.csv')

In [ ]:

df_immig_sample.shape
df_immig_sample.columns
df_immig_sample.head(3)

In [ ]:
df_demographics.shape
df_demographics.columns
df_demographics.head(3)

In [ ]:
df_ariport_Code.shape
df_ariport_Code.columns
df_ariport_Code.head(3)

In [ ]:
df_temperature.shape
df_temperature.columns
df_temperature.head(3)


## Step 2: Explore and Assess the Data
##### Explore and Cleaning the Data 
This step will identify data quality issues, like missing values, duplicate data, etc. After that, I will  remove unnecessary data for clean data.

###  1. Immigration dataset

Show some informations of the immigration dataset.

In [ ]:
df_immigration_sp.count()
df_immigration_sp.show(3)
df_immigration_sp.printSchema()


In [ ]:
#Let find numnber of NULL value for all columns of immigration data table.
df_immigration_sp.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_immigration_sp.columns]).show()

Because of arrdate and depdate columns have double type and it just show indicate the arrival data and departure day so that the double type is unnecessary
I will convert 2 this rows to interger type.

In [ ]:
# Convert for arrival data column
df_imigration_arr= df_immigration_sp.withColumn("arrdate",df_immigration_sp["arrdate"].cast('int'))
df_imigration_arr.printSchema()
df_imigration_arr.head(3)
# Convert for departure data column
df_imigration_dep= df_imigration_arr.withColumn("depdate",df_imigration_arr["depdate"].cast('int'))
df_imigration_dep.printSchema()
df_imigration_dep.head(3)



In [ ]:
# Create a view of the immigration dataset
df_imigration_dep.createOrReplaceTempView("immigration_table")

Check the iccid of the immigration table to see if it can be used as the primary key.

In [ ]:
df_imigration_dep.count()

In [ ]:

spark.sql("""
        SELECT COUNT (DISTINCT cicid)
        FROM immigration_table
""").show()

In [ ]:
#The port code of i94port has a length is 3 so let's check if it can be applied to the codes in the dataset.
spark.sql("""
            SELECT LENGTH (i94port) AS len
            FROM immigration_table
            GROUP BY len
""").show()

Because of SAS correspond to the number of days since 1960-01-01 so I keep dataset from 1960-01-01

In [ ]:
df_immigration = spark.sql("SELECT *,date_add(to_date('1960-01-01'), arrdate) AS arrival_date FROM immigration_table")
df_immigration.createOrReplaceTempView("immigration_table")
df_immigration.show(2)

To make more clearlly of i94visa columms, I replace the data in the I94VISA columns

In [ ]:

df_immigration = spark.sql("""SELECT *, CASE 
                                        WHEN i94visa = 1.0 THEN 'Business' 
                                        WHEN i94visa = 2.0 THEN 'Pleasure'
                                        WHEN i94visa = 3.0 THEN 'Student'
                                        ELSE 'N/A' END AS visa_type
                                    FROM immigration_table""")
df_immigration.createOrReplaceTempView("immigration_table")
df_immigration.show(2)


In [ ]:

df_immigration = spark.sql("""SELECT *, CASE 
                        WHEN depdate >= 1.0 THEN date_add(to_date('1960-01-01'), depdate)
                        WHEN depdate IS NULL THEN NULL
                        ELSE 'N/A' END AS departure_date            
                FROM immigration_table""")
df_immigration.createOrReplaceTempView("immigration_table")
df_immigration.show(2)

Let's check N/A values of arrival_date and departure_date

In [ ]:
spark.sql("SELECT count(*) FROM immigration_table WHERE arrival_date = 'N/A'").show()
spark.sql("SELECT count(*) FROM immigration_table WHERE departure_date = 'N/A'").show()

Check the relationshiop between departure_date and arrival_date

In [ ]:
spark.sql("""
            SELECT COUNT(*)
            FROM immigration_table
            WHERE departure_date <= arrival_date
        """).show()

spark.sql("""
        SELECT arrival_date, departure_date
        FROM immigration_table
        WHERE departure_date <= arrival_date
        """).show()
spark.sql("""
        SELECT *
        FROM immigration_table
        WHERE departure_date >= arrival_date
        """).createOrReplaceTempView("immigration_table")


In [ ]:
#check distinct arrival dates
spark.sql("""
            SELECT COUNT (DISTINCT arrival_date) 
            FROM immigration_table;
        """).show()

In [ ]:
#check distinct departure dates
spark.sql("""
            SELECT COUNT (DISTINCT departure_date) 
            FROM immigration_table;
        """).show()

In [ ]:
#check the common values between the two sets
spark.sql("""   
            SELECT COUNT(DISTINCT departure_date) 
            FROM immigration_table 
            WHERE departure_date 
            IN (SELECT DISTINCT arrival_date FROM immigration_table);
        """).show()

In [ ]:
# Check the data for the various arrival modes
spark.sql("""
            SELECT i94mode, count(*)
            FROM immigration_table
            GROUP BY i94mode
        """).show()

In [ ]:

df_immigration = spark.sql("""SELECT *, CASE 
                                        WHEN i94mode = 1.0 THEN 'Air' 
                                        WHEN i94mode = 2.0 THEN 'Sea'
                                        WHEN i94mode = 3.0 THEN 'Land'
                                        WHEN i94mode = 9.0 THEN 'Not reported'
                                        ELSE 'N/A' END AS arrival_modes
                                    FROM immigration_table""")
df_immigration.createOrReplaceTempView("immigration_table")
df_immigration.show(2)

Due to  our dataset can work with the airports  so I just keep only arrival by Air

In [ ]:
df_immigration = spark.sql("""SELECT *
                                FROM immigration_table
                                WHERE arrival_modes = 'Air' """)
df_immigration.createOrReplaceTempView("immigration_table")
df_immigration.show(2)

In [ ]:
# Let's check N/A values of i94bir to ensure if there are missing values or not.
spark.sql("""
            SELECT COUNT(*)
            FROM immigration_table
            WHERE i94bir IS NULL
        """).show()

In [ ]:
# Check the maximum and minimum values of the biryear
spark.sql("SELECT  MIN(biryear), MAX(biryear) FROM immigration_table WHERE biryear IS NOT NULL").show()

In [ ]:
# frequency of travellers by birth year
spark.sql("""
            SELECT biryear, COUNT(*)
            FROM immigration_table 
            WHERE biryear IS NOT NULL
            GROUP BY biryear
            ORDER BY biryear ASC
        """).show()

In [ ]:
#Since the birth year is available for each row, we can compute the age. Let's check if computed values match the age
spark.sql("""   
            SELECT (2016-biryear)-i94bir AS difference, count(*) 
            FROM immigration_table 
            WHERE i94bir IS NOT NULL 
            GROUP BY difference
        """).show()

In [ ]:
# Check the gender to see if the data is useable
spark.sql("""
                SELECT gender, count(*) 
                FROM immigration_table
                GROUP BY gender
        """).show()

I will filter out all the rows where the gender is missing or incorrect. In this case, I will remove null values,U and A so that the gender just only has 2 values is F: Female and M: Male

In [ ]:
df_immigration =spark.sql("""
            SELECT * FROM immigration_table 
            WHERE gender IN ('F', 'M')""")
df_immigration.createOrReplaceTempView("immigration_table")
df_immigration.show(2)

Let's check N/A values of residence countries,reported address,citizenship countries and visa type

In [ ]:

#residence countries
spark.sql("""
SELECT count(*) 
FROM immigration_table
WHERE i94res IS NULL
""").show()
#citizenship countries
spark.sql("""
SELECT count(*) 
FROM immigration_table
WHERE i94cit IS NULL
""").show()
#reported address
spark.sql("""
SELECT count(*) 
FROM immigration_table
WHERE i94addr IS NULL
""").show()

spark.sql("""
SELECT COUNT(*)
FROM immigration_table
WHERE visatype IS NULL
""").show()



In [ ]:
spark.sql("""
SELECT visa_type, visatype, count(*)
FROM immigration_table
GROUP BY visa_type, visatype
ORDER BY visa_type, visatype
""").show()

In [ ]:

df_immigration = spark.sql("""SELECT * FROM immigration_table""").show(3)


### 2.Temperature data

In [ ]:
df_temperature.shape

In [ ]:

df_temperature.head(5)

In [ ]:
df_temperature['Country'].nunique()

Even though the datafame contains data of 159 countries, this project only needs data of United States. Therefore, I will filter out countries unnecessarily.

In [ ]:

# Keep only data for the United States
df_temperature = df_temperature[df_temperature['Country']=='United States']
df_temperature.head(3)

After the WWII, the commercial air travel began starting in the 1950s so that I will exclude any data prior to 1950s.

In [ ]:
# Remove all dates prior to 1950
df_temperature=df_temperature[df_temperature['dt']>"1950-01-01"]
df_temperature.shape
df_temperature.head(3)

In [ ]:
# Let's check the most recent date in the dataset
df_temperature['dt'].max()

In [ ]:
df_temperature['AverageTemperature'].max()

Let's check for null value

In [ ]:

_null_temperature = df_temperature.isnull().sum()
_null_temperature


In [ ]:
ax = (_null_temperature[_null_temperature>0]/df_temperature.shape[0]*100).plot(kind='bar', title=f"Percent of type of Airport %")
plt.show()

In [ ]:
df_temperature[df_temperature.AverageTemperature.isnull()]
df_temperature[df_temperature.AverageTemperatureUncertainty.isnull()]

Let's make sure the combination of city and date can be used as a primary key

In [ ]:
df_temperature[(df_temperature['City'] == 'Arlington') & (df_temperature.dt == '1950-03-01')]

In [ ]:
df_temperature.shape
df_temperature[['City','dt']].drop_duplicates().shape
df_temperature[df_temperature[['City','dt']].duplicated()].head()
df_temperature[(df_temperature['City'] == 'Arlington') & (df_temperature.dt == '1950-03-01')]

### 3.Airport data


In [ ]:
df_ariport_Code.shape

In [ ]:

df_ariport_Code.head(5)


Extract data of coordinates to 2 parameters is longitude and latitude

In [ ]:
longitude = []
latitude = []
for data in df_ariport_Code['coordinates']:
    arrays= data.split(', ')
    longitude.append(float(arrays[0]))
    latitude.append(float(arrays[1]))

df_ariport_Code.insert(12,'longitude',longitude)
df_ariport_Code.insert(13,'latitude',latitude)
df_ariport_Code.head(5)

Shows the position of air port in World Map

In [ ]:
gdf = geopandas.GeoDataFrame(
    df_ariport_Code[['longitude','latitude']], 
    geometry=geopandas.points_from_xy(df_ariport_Code['longitude'], 
                                      df_ariport_Code['latitude']))
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
ax = world.plot(color='white', edgecolor='black', figsize=(16,12))
gdf.plot(ax=ax, color='green', markersize=0.3)
plt.show()

In [ ]:
# Check the countries where these airports are located
df_ariport_Code.groupby('iso_country')['iso_country'].count()
_df_ariport_count_ = df_ariport_Code.groupby('iso_region')['iso_region'].count()
_df_ariport_count_

Check  there is no missing data in the iso_county field.


In [ ]:
df_ariport_Code[df_ariport_Code['iso_country'].isna()].shape
df_ariport_Code.head()

In [ ]:
# Check the missing country values to see if the continent data is filled out
df_ariport_Code[df_ariport_Code['iso_country'].isna()].groupby('continent')['continent'].count()
df_ariport_Code.head()

In [ ]:
# Since all missing values are in africa, we simly remove them from the dataset
df_ariport_Code = df_ariport_Code[df_ariport_Code.iso_country.fillna('').str.upper().str.contains('US')].copy()
df_ariport_Code.head()

In [ ]:
df_ariport_Code.groupby('type')['type'].count()

The graph to show the percentage of the type of the Aripor

In [ ]:
port_type = df_ariport_Code.groupby('type')['type'].count()
ax = (port_type[port_type>0]/df_ariport_Code.shape[0]*100).plot(kind='bar', title=f"Percent of type of Airport %")
plt.show()

In [ ]:

excludedValues = ['closed', 'heliport', 'seaplane_base', 'balloonport']

Basically, the Project target on the Airport so that some type of port is not correctly such as balloonport,helipori. Therefore, I will make filter out the some type of port that is unnecessary.

In [ ]:

df_airports = df_ariport_Code[~df_ariport_Code['type'].str.strip().isin(excludedValues)].copy()
df_airports.head()

In [ ]:
# We also verify that the municipality field is available for all airports
df_airports[df_airports.municipality.isna()].head(5)
df_airports = df_airports[~df_airports['municipality'].isna()].copy()

In [ ]:
# convert the municipality column to upper case in order to be able to join it with our other datasets.
df_airports.municipality = df_airports.municipality.str.upper()
df_airports.groupby('iso_region')['iso_region'].count()


In [ ]:
# apply len to the iso_region field to see which ones are longer than 5 characters since the field is a combination of US and state code
df_airports['len'] = df_airports["iso_region"].apply(len)
# let's remove the codes that are incorrect.
df_airports = df_airports[df_airports['len']==5].copy()
# finally, let's extract the state code
df_airports['state'] = df_airports['iso_region'].str.strip().str.split("-", n = 1, expand = True)[1]
df_airports.head(5)


In [ ]:

gdf = geopandas.GeoDataFrame(
    df_airports[['longitude','latitude']], 
    geometry=geopandas.points_from_xy(df_airports['longitude'], 
                                      df_airports['latitude']))
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
ax = world.plot(color='white', edgecolor='black', figsize=(16,12))
gdf.plot(ax=ax, color='blue', markersize=0.3)
plt.show()

### 4.Demographic data

In [ ]:
df_demographics.shape

In [ ]:
df_demographics.head(10)

In [ ]:
# Convert the city to upper case and remove any leading and trailing spaces
df_demographics.City = df_demographics.City.str.upper().str.strip()
df_demographics.head(10)

In [ ]:
# Check the missing values
df_demographics.isnull().sum()

In [ ]:

df_demographics.groupby('City')['City'].count()

In [ ]:
# remove any leading or trailing spaces and convert to upper case
df_demographics.City = df_demographics.City.str.strip().str.upper()

In [ ]:
#primary key will be the combination of city name and race
df_demographics[df_demographics[['City','Race']].duplicated()].head()

In [ ]:

df_demographics[(df_demographics.City == 'WILMINGTON') & (df_demographics.Race == 'Asian')]
df_demographics[df_demographics[['City', 'State','Race']].duplicated()].head()

In [ ]:
_demographics_null =  df_demographics.isnull().sum()
ax = (_demographics_null[_demographics_null>0]/df_demographics.shape[0]*100).plot(kind='bar', title=f"Percent Null of demographics %")
plt.show()

In [ ]:
df_demographics

## Step 3: Define the Data Model

### 3.1 Conceptual Data Model

I create a Star Schema for OLAP queries. The Schema for each table will following the bellow figure:

![Star Schema](./documents/i94_immagration.png "Star Schema")

### 3.2 Mapping Out Data Pipelines

Extract the I94_SAS_Labels_Descriptions

Extract the the I94_SAS_Labels_Description.SAS file to CSV files.
The output files:
- i94_country.csv
- i94_model.csv
- i94_port.csv
- i94_state_addrl.csv
- i94_visa.csv


Data loading,transformation,cleanup and create spark data table frame

##### fact_immagration


- Load the immigration data from the base dataset.
- Convert for arrival and departure data column.
- Convert for i94visa and i94mode data column.
- Remove all entries into the united states that weren't via air travel.
- Drop rows where the gender values entered is undefined.
- Convert the arrival dates and departure dates into new columns.
- Filter up the arrival modes.
- Use an inner join to drop invalid codes country of citizenship and for countr of residence.
- Add entry_port names and entry port states.
- Compute the age of each individual.
- Insert the immigration fact data into a spark dataframes.
- Saving the data in parquet format.

#### dim_airports

- Load the csv directly into a spark dataframes.
- Equivalent to the following pandas code.
- Verify that the municipality field is available for all airports.
- Convert the municipality column to upper case in order to be able to join it with our other datasets.
- Extract the state codes.
- Convert the dataframes from pandas to spark.
- Saving the data in parquet format in a spark dataframe.

#### dim_demographics

- Load the various csv files into pandas dataframes.
- Convert the city to upper case and remove any leading and trailing spaces.
- Remove any leading or trailing spaces and convert to upper case.
- Primary key will be the combination of city name and race.
- Convert the dataframes from pandas to spark.
- Insert data into the demographics dim table.
- Saving the data in parquet format.


#### dim_temperture

- Load data base abd use pandas to load DataFame.
- Keep only data for the United States.
- Convert the date to datetime objects.
- Remove all dates prior to 1950.
- Convert the city names to upper case.
- Convert the dataframes from pandas to spark
- Insert the temperature dim data into a spark dataframe.
- Saving the data in parquet format.

## Step 4: Run Pipelines to Model the Data 
### 4.1 Create the data model
Build the data pipelines to create the data model. Refer and run etl.py script to create the data pipelines.

In [ ]:
from etl import *
print("Start the ETL process ......")
print("----------------------------------------------------------------")
print("Create the Spark Session...")
spark = create_Spark()
print("----------------------------------------------------------------")
print("Extraxting SAS dataset...")
get_SAS_dataset()
print("----------------------------------------------------------------")
main_etl(spark)
print("Finish the ETL process")



### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
from quality_check import *
print("Start the data quality_check process ......")
print("----------------------------------------------------------------")
print("Create the Spark Session...")
spark = create_Spark()
print("----------------------------------------------------------------")
main_check(spark)
    

### 4.3 Data dictionary 


##### fact_immagration


<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>  
 <tr><td class="tg-0pky">cicid</td><td class="tg-0pky">Record ID</td></tr>
 <tr><td class="tg-0pky">citizenship_country</td><td class="tg-0pky"> The code for immigrant city of residence</td></tr>
 <tr><td class="tg-0pky">residence_country</td><td class="tg-0pky">The code for immigrant country of residence</td></tr>
 <tr><td class="tg-0pky">city</td><td class="tg-0pky">The city name of arrival</td></tr>
 <tr><td class="tg-0pky">state</td><td class="tg-0pky">US state of arrival</td></tr>
 <tr><td class="tg-0pky">arrival_date</td><td class="tg-0pky">Arrival Date in the USA</td></tr>
 <tr><td class="tg-0pky">departure_date</td><td class="tg-0pky">Departure Date from the USA </td></tr>
 <tr><td class="tg-0pky">age</td><td class="tg-0pky"> Age of Respondent in Years </td></tr>
 <tr><td class="tg-0pky">visa_type</td><td class="tg-0pky">	The codes of VISA</td> </tr>
 <tr><td class="tg-0pky">detailed_visa_type</td><td class="tg-0pky">The detailed about type of VISA</td></tr>
 <tr><td class="tg-0pky">gender</td><td class="tg-0pky">The immigrant sex</td></tr>
</table>

##### dim_demographics

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">city</td><td class="tg-0pky">City Name</td>
 <tr><td class="tg-0pky">state</td><td class="tg-0pky">US State where city is located</td>
 <tr><td class="tg-0pky">median_age</td><td class="tg-0pky">Median age of the population</td>
 <tr><td class="tg-0pky">male_population</td><td class="tg-0pky">Count of male population</td>
 <tr><td class="tg-0pky">female_population</td><td class="tg-0pky">Count of female population</td>
 <tr><td class="tg-0pky">total_population</td><td class="tg-0pky">Count of total population</td>
 <tr><td class="tg-0pky">number_of_veterans</td><td class="tg-0pky">Count of total Veterans</td>
 <tr><td class="tg-0pky">toreign born</td><td class="tg-0pky">Count of residents of the city that were not born in the city</td>
 <tr><td class="tg-0pky">average_household_size</td><td class="tg-0pky">Average city household size</td>
 <tr><td class="tg-0pky">state_code</td><td class="tg-0pky">US state code </td>
 <tr><td class="tg-0pky">race</td><td class="tg-0pky">Respondent race</td>
 <tr><td class="tg-0pky">count</td><td class="tg-0pky">Count of city's individual per race</td>
</table>

##### dim_airports

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
<tr><td class="tg-0pky">ident</td><td class="tg-0pky">Airport identity number </td></tr>
<tr><td class="tg-0pky">type</td><td class="tg-0pky">Type of airport by size</td></tr>
<tr><td class="tg-0pky">name</td><td class="tg-0pky"> Airport name</td></tr>    
<tr><td class="tg-0pky">elevation_ft</td><td class="tg-0pky"> Elevation of airport in feet</td></tr>
<tr><td class="tg-0pky">iso_country</td><td class="tg-0pky">Country of airport </td></tr>
<tr><td class="tg-0pky">iso_region</td><td class="tg-0pky">Region of airport within country </td></tr>
<tr><td class="tg-0pky">municipality</td><td class="tg-0pky"> Municipality of airport </td></tr>
<tr><td class="tg-0pky">gps_code</td><td class="tg-0pky">GPS code </td></tr>
<tr><td class="tg-0pky">iata_code</td><td class="tg-0pky">IATA code  </td></tr>
<tr><td class="tg-0pky">local_code</td><td class="tg-0pky">Local identity code  </td></tr>
<tr><td class="tg-0pky">coordinates</td><td class="tg-0pky">Longitude and Latitude of airport </td></tr>
<tr><td class="tg-0pky">len</td><td class="tg-0pky">Length of the ident </td></tr>
<tr><td class="tg-0pky">state</td><td class="tg-0pky">State code </td></tr>

</table>

##### dim_temperture

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
<tr><td class="tg-0pky">date</td><td class="tg-0pky">Data time </td></tr>
 <tr><td class="tg-0pky">city</td><td class="tg-0pky">City name</td></tr>
 <tr><td class="tg-0pky">average_temperature</td><td class="tg-0pky">Average temperature of city</td></tr>    
 <tr><td class="tg-0pky">average_termperature_uncertainty</td><td class="tg-0pky">Uncertainty of Avg Temp of city</td></tr>
</table>

#### Step 5: Complete Project Write Up


* The rationale for the choice of tools and technologies for the project:
    - Pandas for loading CSV data files then process,clean and analyse data
    - Spark for large datasets such as sas7bdat data. It's helpful for processing to extract,transform,load and store tables.
    
    

* Propose how often the data should be updated:
    - Because the raw data of immigration and temperature are built based on the month so that dataset should be update every month.
    - For all tables during the update, it should proceed with "append" mode.
* How often ETL script should be run:
    - The ETL script should be run monthly basis

* Suggestions for following scenarios:
    * Data is 100x:
        - The data would be stored in AWS S3 bucket or orther cloud storage.
        - Spark platform can still use to process the data but it should enable parallel processing of the data.
        - The AWS Redshift is good choose to store the data during processing of the ETL script running.
    * Data is used in dashboard and updated every day 07:00AM:
        - Use the Apache Airflow to perform the ETL and data qualtiy validation.
        - The output data should be store and updated in cloud storage susch as AWS RDS to allow the dashboard to display the data all the time.
    * The database needed to be accessed by 100+ people:
        - In this case, the data should be migrated to the AWS Redshift to allow the auto-scaling capabilities.
        - Immigration the Elastic Load Balancing of AWS to improve the performance of the dashboard application.